In [1]:
import os
import shutil
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.io import wavfile
import librosa
import subprocess

import sys
sys.path.insert(1, '../src/')
from drum_audio import DrumAudio

In [2]:
def load_model(model_path):
  return tf.keras.models.load_model(model_path)

In [3]:
CY_model = load_model('../models/CY_60epochs')
HH_model = load_model('../models/HH_ResNet')
KD_model = load_model('../models/KD_ResNet')
SD_model = load_model('../models/SD_ResNet')

In [4]:
audio = DrumAudio('../data/drums_audio/MusicDelta_80sRock_Drum.wav')

In [5]:
transcription = audio.predict_kits(CY_model,HH_model,KD_model,SD_model)

In [6]:
transcription

,time,index_position,CY,HH,KD,SD
0,0.000000,0,0,0,1,0
1,0.554989,24475,0,0,1,1
2,1.103810,48678,0,0,1,0
3,1.657279,73086,0,0,1,1
4,2.228571,98280,0,0,1,0
5,2.777052,122468,0,0,1,1
6,3.315986,146235,0,0,1,0
7,3.831859,168985,0,0,1,1
8,4.373129,192855,1,0,1,0
9,4.880884,215247,0,0,1,1


In [7]:
audio.get_lilypond_transcription(transcription)

In [9]:
audio.get_pdf_transcription()